В этом ноубуке используются обeченные тестовые модели, с их помощью происходит сэмплирование и далее закодированные категориальные переменные переводятся в изначальные. После чего замеряется качество.

Способ нормализации: quantile transformation для числовых признаков и standart scale для категориальных OHE

Добавление шума: ко всем признакам

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ON_QnSC.tabddpm_ON_QnSC import *

from tqdm.notebook import tqdm

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

### Подготовка

In [3]:
dataname = 'shoppers_ON_QnSC_100k'
dataname_initial = 'shoppers'
model_name = 'tabddpm_ON_QnSC'
model_name_postfix = '_100k'

In [4]:
if torch.cuda.is_available():
    device = f'cuda'
else:
    device = 'cpu'
CONFIG.add_arg('device', device)
print(f"Device: {device}")

CONFIG.add_arg('dataname', dataname)
CONFIG.add_arg('real_path',
                   f'./synthetic/{dataname}/initial_real.csv')
CONFIG.add_arg('test_path',
                   f'./synthetic/{dataname}/initial_test.csv')

CONFIG.add_arg('info_path',
                   f'./data/{dataname}/initial_info.json')

Device: cuda


In [5]:
model_save_path_hist = []
idx = [5, 6]

for sigma in [0, 0.01, 0.25]:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_const_{str(sigma).replace('.', '_')}"
    model_save_path_hist.append(model_save_path)

for i in idx:
    model_save_path=f"./models/{model_name}/ckpt/{dataname_initial}/{model_name}{model_name_postfix}_mult_{i}"
    model_save_path_hist.append(model_save_path)

In [6]:
from pickle import dump, load
with open(f'data/{dataname}/normalizers.json', 'r') as f:
        normalizers = json.load(f)
normalizers['num_normalizer'] = load(open(f'./data/{dataname}/num_normalizer_{dataname}.pkl', 'rb'))
normalizers['cat_normalizer'] = load(open(f'./data/{dataname}/cat_normalizer_{dataname}.pkl', 'rb'))
print(f"normalizers received from `data/{dataname}` folder")

normalizers received from `data/shoppers_ON_QnSC_100k` folder


In [7]:
normalizers

{'len_num_prev': 10,
 'len_cat_prev': 65,
 'len_target_prev': 1,
 'num_normalizer': QuantileTransformer(n_quantiles=369, output_distribution='normal',
                     random_state=0, subsample=1000000000),
 'cat_normalizer': StandardScaler()}

### Sample + Eval

In [8]:
CONFIG.get_all_args()

{'dataname': 'shoppers_ON_QnSC_100k',
 'method': 'tabddpm',
 'device': 'cuda',
 'mode': 'train',
 'train': 1,
 'sample_save_path': 'synthetic/shoppers/tabddpm.csv',
 'sigma_scheduller_name': 'constant',
 'sigma_value': 0.001,
 'num_noise': 1,
 'real_path': './synthetic/shoppers_ON_QnSC_100k/initial_real.csv',
 'test_path': './synthetic/shoppers_ON_QnSC_100k/initial_test.csv',
 'info_path': './data/shoppers_ON_QnSC_100k/initial_info.json',
 'save_path': './synthetic/magic_ON_QnSC_100k/initial_tabddpm_ON_QnSC_100k_mult_6.csv',
 'num_clusters': 25,
 'save_cat': None}

In [9]:
model_save_path_hist

['./models/tabddpm_ON_QnSC/ckpt/shoppers/tabddpm_ON_QnSC_100k_const_0',
 './models/tabddpm_ON_QnSC/ckpt/shoppers/tabddpm_ON_QnSC_100k_const_0_01',
 './models/tabddpm_ON_QnSC/ckpt/shoppers/tabddpm_ON_QnSC_100k_const_0_25',
 './models/tabddpm_ON_QnSC/ckpt/shoppers/tabddpm_ON_QnSC_100k_mult_5',
 './models/tabddpm_ON_QnSC/ckpt/shoppers/tabddpm_ON_QnSC_100k_mult_6']

In [10]:
overall_metrics = {}

for model_save_path in model_save_path_hist:
    model_name = model_save_path.split('/')[-1]
    sample_save_path = f'./synthetic/{dataname}/{model_name}.csv'

    CONFIG.add_arg('method', model_name)
    CONFIG.add_arg('sample_save_path',
                       f"synthetic/{CONFIG.get_arg('dataname')}/{CONFIG.get_arg('method')}.csv")

    overall_metrics[model_name] = {}

    # print(model_save_path, model_name, sample_save_path)

    # tabddpm_noise_ohe = TabDDPM_OHE_Noise_QnSC(CONFIG, model_save_path=model_save_path, sigmas=None)
    # tabddpm_noise_ohe.sample(sample_save_path=sample_save_path)
    # postsample_OHE(dataname, f'./synthetic/{dataname}/initial_{model_name}.csv', normalizers=normalizers)

    CONFIG.add_arg('sample_save_path',
                       f'./synthetic/{dataname}/initial_{model_name}.csv')

    # подсчет метрик
    try:
        overall_metrics[model_name]['base_metrics'] = calculate_base_metrics(make_binary=False) # ошибка
    except Exception as e:
        overall_metrics[model_name]['base_metrics'] = np.nan
    overall_metrics[model_name]['similarity'] = calculate_similarity()
    overall_metrics[model_name]['mle'] = calculate_mle()
    overall_metrics[model_name]['detection'] = calculate_detection()
    overall_metrics[model_name]['DCR'] = calculate_DCR()
    overall_metrics[model_name]['quality'] = calculate_alpha_beta()

/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/venv/main/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

TABDDPM_ON_QNSC_100K_CONST_0: SHOPPERS_ON_QNSC_100K Dataset
Original Logistic: 0.621
Synthetic Logistic: 0.634
Original Tree: 0.570
Synthetic Tree: 0.551 

TABDDPM_ON_QNSC_100K_CONST_0 Accuracy Loss: SHOPPERS_ON_QNSC_100K
Logistic: -2.156%
Tree: 3.330% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 18/18 [00:00<00:00, 159.68it/s]|
Column Shapes Score: 98.41%

(2/2) Evaluating Column Pair Trends: |██████████| 153/153 [00:01<00:00, 128.02it/s]|
Column Pair Trends Score: 97.61%

Overall Score (Average): 98.01%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_100K_CONST_0: 1.588 ± 0.765
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_100K_CONST_0: 2.387 ± 1.391


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:06<00:00,  5.78it/s]


...

TABDDPM_ON_QNSC_100K_MULT_6: SHOPPERS_ON_QNSC_100K Dataset
Original Logistic: 0.621
Synthetic Logistic: 0.257
Original Tree: 0.570
Synthetic Tree: 0.233 

TABDDPM_ON_QNSC_100K_MULT_6 Accuracy Loss: SHOPPERS_ON_QNSC_100K
Logistic: 58.651%
Tree: 59.078% 

CALCULATING COLUMN-WISE DENSITY ESTIMATION & PAIR-WISE COLUMN CORRELATION...
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 18/18 [00:00<00:00, 345.53it/s]|
Column Shapes Score: 61.37%

(2/2) Evaluating Column Pair Trends: |██████████| 153/153 [00:00<00:00, 172.79it/s]|
Column Pair Trends Score: 60.5%

Overall Score (Average): 60.93%

Error rate (%) of column-wise density estimation TABDDPM_ON_QNSC_100K_MULT_6: 38.634 ± 14.860
Error rate (%) of pair-wise column correlation score TABDDPM_ON_QNSC_100K_MULT_6: 39.504 ± 21.193


DONE! 

DRAW COLUMN VALUES DISTRIBUTIONS...


DONE!


100%|██████████| 36/36 [00:06<00:00,  5.55it/s]


Saving scores to  eval/mle/shoppers_ON_QnSC_100k/tabddpm_ON_QnSC_100k_mult_6.json
ROC - AUC обучения на синтетических данных TABDDPM_ON_QNSC_100K_MULT_6: 0.508 ± 0.010
shoppers_ON_QnSC_100k, tabddpm_ON_QnSC_100k_mult_6: 0.0020084387951287974
n_clusters: 25
DCR Score, a value closer to 0.5 is better
shoppers_ON_QnSC_100k-tabddpm_ON_QnSC_100k_mult_6, DCR Score = 0.501757231684239
=========== All Features ===========
Data shape:  (11097, 77)
alpha precision: 0.020522, beta recall: 0.000889


In [11]:
final_metrics_table = []
model_short = 'tabddpm_ON_QnSC_100k'
for m in overall_metrics.keys():
    # Сбор таблички результатов
    tmp = pd.DataFrame([{'Model':model_short, 'Type':m, 'Data':dataname}])
    tmp.columns = pd.MultiIndex.from_tuples([('', i) for i in tmp.columns])
    result = [tmp]
    
    for metric_group in overall_metrics[m].keys():
        tmp = pd.DataFrame([overall_metrics[m][metric_group]])
        tmp.columns = pd.MultiIndex.from_tuples([(metric_group, i) for i in tmp.columns])
        result.append(tmp)
    result = pd.concat(result, axis = 1)
    final_metrics_table.append(result)
    
final_metrics_table = pd.concat(final_metrics_table)

In [12]:
final_metrics_table#.drop(columns=('base_metrics', 0))

\
                  Model                             Type   
0  tabddpm_ON_QnSC_100k     tabddpm_ON_QnSC_100k_const_0   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_01   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_25   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_5   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_6   

                              base_metrics                                   \
                    Data Original Logistic Synthetic Logistic Original Tree   
0  shoppers_ON_QnSC_100k          0.620525           0.633907          0.57   
0  shoppers_ON_QnSC_100k          0.620525           0.630072          0.57   
0  shoppers_ON_QnSC_100k          0.620525           0.629540          0.57   
0  shoppers_ON_QnSC_100k          0.620525           0.639423          0.57   
0  shoppers_ON_QnSC_100k          0.620525           0.256579          0.57   

                                                                  \
  Synthetic Tree Accuracy Loss Logistic, % Accuracy Loss Tree, %   
0       0.551020                 -2.156492              3.329753   
0       0.526829                 -1.538462              7.573813   
0       0.508314                 -1.452785             10.822186   
0       0.572193                 -3.045488             -0.384651   
0       0.233256                 58.651316             59.077833   

              similarity  ...  \
  Column Shapes Score, %  ...   
0              98.412484  ...   
0              98.417991  ...   
0              97.735124  ...   
0              98.362420  ...   
0              61.365935  ...   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                            1.587516   
0                                            1.582009   
0                                            2.264876   
0                                            1.637580   
0                                           38.634065   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                0.764767   
0                                                0.894148   
0                                                1.095060   
0                                                0.890355   
0                                               14.860456   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                2.387228   
0                                                2.438910   
0                                                2.816406   
0                                                2.718609   
0                                               39.504177   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                    1.390743   
0                                                    1.438104   
0                                                    1.508141   
0                                                    1.789526   
0                                                   21.192926   

                                         mle  \
  ROC - AUC обучения на синтетических данных   
0                                   0.936284   
0                                   0.895920   
0                                   0.895389   
0                                   0.908483   
0                                   0.508245   

                                                  detection       DCR  \
  ROC - AUC обучения на синтетических данных, std     Score     Score   
0                                        0.003027  0.938076  0.510679   
0                                        0.005397  0.953931  0.496531   
0                                

In [13]:
pd.set_option('display.max_columns', None)
final_metrics_table.round(4).sort_values([('similarity', 'Overall Score (Average), %')],
                                                                          ascending=False)

\
                  Model                             Type   
0  tabddpm_ON_QnSC_100k     tabddpm_ON_QnSC_100k_const_0   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_01   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_5   
0  tabddpm_ON_QnSC_100k  tabddpm_ON_QnSC_100k_const_0_25   
0  tabddpm_ON_QnSC_100k      tabddpm_ON_QnSC_100k_mult_6   

                              base_metrics                                   \
                    Data Original Logistic Synthetic Logistic Original Tree   
0  shoppers_ON_QnSC_100k            0.6205             0.6339          0.57   
0  shoppers_ON_QnSC_100k            0.6205             0.6301          0.57   
0  shoppers_ON_QnSC_100k            0.6205             0.6394          0.57   
0  shoppers_ON_QnSC_100k            0.6205             0.6295          0.57   
0  shoppers_ON_QnSC_100k            0.6205             0.2566          0.57   

                                                                  \
  Synthetic Tree Accuracy Loss Logistic, % Accuracy Loss Tree, %   
0         0.5510                   -2.1565                3.3298   
0         0.5268                   -1.5385                7.5738   
0         0.5722                   -3.0455               -0.3847   
0         0.5083                   -1.4528               10.8222   
0         0.2333                   58.6513               59.0778   

              similarity                              \
  Column Shapes Score, % Column Pair Trends Score, %   
0                98.4125                     97.6128   
0                98.4180                     97.5611   
0                98.3624                     97.2814   
0                97.7351                     97.1836   
0                61.3659                     60.4958   

                              \
  Overall Score (Average), %   
0                    98.0126   
0                    97.9895   
0                    97.8219   
0                    97.4594   
0                    60.9309   

                                                       \
  Error rate (%) of column-wise density estimation, %   
0                                              1.5875   
0                                              1.5820   
0                                              1.6376   
0                                              2.2649   
0                                             38.6341   

                                                           \
  Error rate (%) of column-wise density estimation std, %   
0                                                  0.7648   
0                                                  0.8941   
0                                                  0.8904   
0                                                  1.0951   
0                                                 14.8605   

                                                           \
  Error rate (%) of pair-wise column correlation score, %   
0                                                  2.3872   
0                                                  2.4389   
0                                                  2.7186   
0                                                  2.8164   
0                                                 39.5042   

                                                               \
  Error rate (%) of pair-wise column correlation score std, %   
0                                                      1.3907   
0                                                      1.4381   
0                                                      1.7895   
0                                                      1.5081   
0                                                     21.1929   

                                         mle  \
  ROC - AUC обучения на синтетических данных   
0                                     0.9363   
0                                     0.8959   
0                                     0.9085   
0                                   

In [14]:
model_name_short = 'tabddpm_ON_QnSC'

In [15]:
f'./eval/total/{dataname}/{model_name_short}{model_name_postfix}_final_metrcs_table.csv'

'./eval/total/shoppers_ON_QnSC_100k/tabddpm_ON_QnSC_100k_final_metrcs_table.csv'

In [16]:
import os
if not os.path.exists(f'./eval/total/{dataname}'):
    os.makedirs(f'./eval/total/{dataname}')
final_metrics_table.to_csv(f'./eval/total/{dataname}/{model_name_short}{model_name_postfix}_final_metrcs_table.csv', index=False)